In [115]:
from sklearn import svm
# from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestClassifier
import matplotlib.pyplot as plt
from nltk.stem import RSLPStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.metrics import classification_report, confusion_matrix

import pandas as pd
import csv
import numpy as np
# Import supplementary visualization code visuals.py
import visuals as vs

import re

In [116]:
!ls

RandomForest.pickle         main.pyc
__pycache__                 model.ipynb
analysis.Rmd                playlist_processor.py
analysis.pdf                protest.csv
analysis_files              pt_stopwords.txt
brz_dictatorship.csv        visuals.py
brz_dictatorship_target.csv whosampled_scraper.py
jovem_guarda.csv


In [162]:
df = pd.read_csv('brz_dictatorship.csv', encoding='utf-8')
df = df.dropna(axis=0)
display(df.head())

stopwords = open("pt_stopwords.txt").readlines()
stopwords = list(map(str.strip, stopwords))

,Unnamed: 0,song_sp_uri,song_name,song_isrc,song_popularity,song_lyrics,class,danceability,energy,key,...,valence,tempo,id,duration_ms,time_signature,artist_genres,artist_name,artist_photo,artist_popularity,artist_sp_followers
0,0,spotify:track:0VUgbCK0k8QWGpLiEV8YYZ,Cálice,BRPGD7800015,50,"pai , afasta mim calice pai , afasta mim calic...",Protest,0.596,0.372,4,...,0.293,123.125,0VUgbCK0k8QWGpLiEV8YYZ,241867,4,bossa nova,Chico Buarque,https://i.scdn.co/image/c1a6ae9e79561abeb0bd97...,63,542214
1,1,spotify:track:2GAFZG9Z7UGS1iMm4Idrnr,Apesar De Você,BRPGD7800024,57,amanha vai outro dia amanha vai outro dia aman...,Protest,0.568,0.574,4,...,0.680,107.820,2GAFZG9Z7UGS1iMm4Idrnr,235547,4,bossa nova,Chico Buarque,https://i.scdn.co/image/c1a6ae9e79561abeb0bd97...,63,542214
2,2,spotify:track:06ND7qqsmIRCuWdQNQIlTE,Roda-Viva,BRSGL6800006,53,dias gente sente partiu morreu gente estancou ...,Protest,0.502,0.512,10,...,0.651,133.174,06ND7qqsmIRCuWdQNQIlTE,233400,4,bossa nova,Chico Buarque,https://i.scdn.co/image/c1a6ae9e79561abeb0bd97...,63,542214
3,3,spotify:track:3Ig3k8enaBZy7cohqfnWVz,Como nossos pais,BRWMB9705419,27,nao quero falar grande amor coisas aprendi dis...,Protest,0.463,0.337,8,...,0.269,96.559,3Ig3k8enaBZy7cohqfnWVz,280627,4,bossa nova,Belchior,https://i.scdn.co/image/22365c7789eb990937bb71...,57,299597
4,4,spotify:track:3AQg8DR76bH9Ko3mZXuSWK,Mosca Na Sopa,BRMCA7300117,45,mosca pousou sopa mosca pintou pra abusar mosc...,Protest,0.609,0.760,2,...,0.880,92.022,3AQg8DR76bH9Ko3mZXuSWK,239187,4,brazilian psychedelic,Raul Seixas,https://i.scdn.co/image/ad67b61dd80b071a2ac908...,63,829961


In [163]:
stemmer = RSLPStemmer()
df['cleaned'] = df['song_lyrics'].astype('str')
df['cleaned'] = df['cleaned'].apply(lambda x: " ".join([stemmer.stem(i) for i in re.sub("[^a-zA-Z]", " ", x).split()]).lower())

In [164]:
vectorizer = TfidfVectorizer(min_df= 3, stop_words=stopwords, sublinear_tf=True, norm='l2', ngram_range=(1, 2))
final_features = vectorizer.fit_transform(df['cleaned']).toarray()
final_features.shape

(184, 1391)

In [165]:
from sklearn.base import BaseEstimator, TransformerMixin

class ItemSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, df, y=None):
        return self

    def transform(self, df, y=None):
        return df[self.key]

class NumSelector(BaseEstimator, TransformerMixin):
    def __init__(self, key):
        self.key = key

    def fit(self, df, y=None):
        return self

    def transform(self, df, y=None):
        return df[self.key].values.reshape(-1, 1)


In [174]:
# install machine learning package
import sys
!{sys.executable} -m pip install sklearn pandas numpy gensim tabulate

In [188]:
from tabulate import tabulate

from gensim.models.word2vec import Word2Vec
from collections import Counter, defaultdict
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score

X = df[['song_lyrics', 'valence', 'duration_ms', 'danceability', 'acousticness', 'tempo']]
y = df['class']

# train word to vector on texts 
model = Word2Vec(X['song_lyrics'], size=100, window=5, min_count=5, workers=2)
w2v = {w: vec for w, vec in zip(model.wv.index2word, model.wv.syn0)}


('lyrics', Pipeline([
                ('selector', ItemSelector(key='song_lyrics')),
                ('tfidf', vectorizer),
            ])),

# multiple machine learning models
mult_nb = Pipeline([('lyrics', Pipeline([
                ('selector', ItemSelector(key='song_lyrics')),
                ("count_vectorizer", CountVectorizer(analyzer=lambda x: x)),
            ])), ("multinomial nb", MultinomialNB())])
bern_nb = Pipeline([("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), ("bernoulli nb", BernoulliNB())])
mult_nb_tfidf = Pipeline([("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), ("multinomial nb", MultinomialNB())])
bern_nb_tfidf = Pipeline([("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), ("bernoulli nb", BernoulliNB())])
# SVM 
svc = Pipeline([("count_vectorizer", CountVectorizer(analyzer=lambda x: x)), ("linear svc", SVC(kernel="linear"))])
svc_tfidf = Pipeline([("tfidf_vectorizer", TfidfVectorizer(analyzer=lambda x: x)), ("linear svc", SVC(kernel="linear"))])

class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.dim = len(word2vec.values())
    
    def fit(self, X, y):
        return self 

    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec] 
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])
    
# and a tf-idf version of the same
class TfidfEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        self.word2weight = None
        self.dim = len(word2vec.values())

    def fit(self, X, y):
        tfidf = TfidfVectorizer(analyzer=lambda x: x)
        tfidf.fit(X)
        # if a word was never seen - it must be at least as infrequent
        # as any of the known words - so the default idf is the max of 
        # known idf's
        max_idf = max(tfidf.idf_)
        self.word2weight = defaultdict(
            lambda: max_idf,
            [(w, tfidf.idf_[i]) for w, i in tfidf.vocabulary_.items()])

        return self

    def transform(self, X):
        return np.array([
                np.mean([self.word2vec[w] * self.word2weight[w]
                         for w in words if w in self.word2vec] or
                        [np.zeros(self.dim)], axis=0)
                for words in X
            ])



etree_w2v = Pipeline([("word2vec vectorizer", MeanEmbeddingVectorizer(w2v)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=200))])
etree_w2v_tfidf = Pipeline([("word2vec vectorizer", TfidfEmbeddingVectorizer(w2v)), 
                        ("extra trees", ExtraTreesClassifier(n_estimators=200))])
all_models = [
    ("mult_nb", mult_nb),
    ("mult_nb_tfidf", mult_nb_tfidf),
    ("bern_nb", bern_nb),
    ("bern_nb_tfidf", bern_nb_tfidf),
    ("svc", svc),
    ("svc_tfidf", svc_tfidf),
    ("w2v", etree_w2v),
    ("w2v_tfidf", etree_w2v_tfidf),
]

## Comparing different models
scores = sorted([(name, cross_val_score(model, X, y, cv=5).mean()) 
                 for name, model in all_models])
print(tabulate(scores, floatfmt=".4f", headers=("model", 'score')))

/Users/gabrielsaruhashi/anaconda2/envs/py37/lib/python3.6/site-packages/ipykernel_launcher.py:20: DeprecationWarning: Call to deprecated `syn0` (Attribute will be removed in 4.0.0, use self.wv.vectors instead).
/Users/gabrielsaruhashi/anaconda2/envs/py37/lib/python3.6/site-packages/sklearn/model_selection/_validation.py:542: FutureWarning: From version 0.22, errors during fit will result in a cross validation score of NaN by default. Use error_score='raise' if you want an exception raised or error_score=np.nan to adopt the behavior from version 0.22.
  FutureWarning)


ValueError: Found input variables with inconsistent numbers of samples: [6, 146]

In [173]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import FeatureUnion
from sklearn.naive_bayes import GaussianNB
from sklearn.decomposition import TruncatedSVD

# this block is to split the dataset into training and testing set 
X = df[['song_lyrics', 'valence', 'duration_ms', 'danceability', 'acousticness', 'tempo']]
Y = df['class']
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.25)

pipeline = Pipeline([
    ('union', FeatureUnion(
        transformer_list=[

            # Pipeline for pulling features from the post's subject line
            ('lyrics', Pipeline([
                ('selector', ItemSelector(key='song_lyrics')),
                ('tfidf', vectorizer),
            ])),
            # Pipeline for standard bag-of-words model for body
            ('duration', Pipeline([
                ('selector', NumSelector(key='duration_ms')),
                ('minxMax', StandardScaler()),
            ])),
            ('tempo', Pipeline([
                ('selector', NumSelector(key='tempo')),
                ('minxMax', StandardScaler()),
            ])),
        ]
    )),
    # Use a SVC classifier on the combined features
    ('clf', RandomForestClassifier()),
])

# fitting our model and save it in a pickle for later use
model = pipeline.fit(X_train, y_train)
# with open('RandomForest.pickle', 'wb') as f:
#     pickle.dump(model, f)

ytest = np.array(y_test)

# confusion matrix and classification report(precision, recall, F1-score)
print(classification_report(ytest, model.predict(X_test)))
print(confusion_matrix(ytest, model.predict(X_test)))


              precision    recall  f1-score   support

Jovem Guarda       0.73      0.88      0.80        25
     Protest       0.81      0.62      0.70        21

   micro avg       0.76      0.76      0.76        46
   macro avg       0.77      0.75      0.75        46
weighted avg       0.77      0.76      0.76        46

[[22  3]
 [ 8 13]]


/Users/gabrielsaruhashi/anaconda2/envs/py37/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/gabrielsaruhashi/anaconda2/envs/py37/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/gabrielsaruhashi/anaconda2/envs/py37/lib/python3.6/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/gabrielsaruhashi/anaconda2/envs/py37/lib/python3.6/site-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype int64 was converted to float64 by StandardScaler.
  warnings.warn(msg, DataConversionWarning)
/Users/g

In [ ]:
def train_network(net, criterion, optimizer, trainloader):
    for epoch in range(2):
        
        for batch_idx, (images, labels) in enumerate(mnist_trainloader):
#             images, labels = data
            
#             print(images.shape)
            outputs = mnist_net.forward(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            if (batch_idx) % 2000 == 0:
                print ('Epoch [{}/{}], Loss: {:.4f}'.format(epoch+1, batch_idx, loss.item())